## 导入模型

In [2]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)

X_train = []
X_test = []

for filename in ["feature_resnet50.h5", "feature_xception.h5", "feature_inception_v3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

## 构建模型

In [3]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## 模型可视化

In [ ]:
from IPython.display import SVG,display
from keras.utils.vis_utils import model_to_dot
import pydot

display(SVG(model_to_dot(model).create(prog='dot', format='svg')))

## 训练

每次epoch保存较好的模型权重进行下一步的训练

In [13]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath="fine_weight.h5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min',
                             save_weights_only=True)

model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True, callbacks=[checkpoint])
model.save_weights("merge_fine_weight.h5")

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 58s 3ms/step - loss: 0.0674 - acc: 0.9768 - val_loss: 0.0203 - val_acc: 0.9940

Epoch 00001: val_loss improved from inf to 0.02034, saving model to mergenet-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 1s 43us/step - loss: 0.0218 - acc: 0.9926 - val_loss: 0.0159 - val_acc: 0.9946

Epoch 00002: val_loss improved from 0.02034 to 0.01594, saving model to mergenet-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 1s 42us/step - loss: 0.0184 - acc: 0.9940 - val_loss: 0.0156 - val_acc: 0.9942

Epoch 00003: val_loss improved from 0.01594 to 0.01562, saving model to mergenet-best_weight.h5
Epoch 4/20
20000/20000 [==============================] - 1s 41us/step - loss: 0.0142 - acc: 0.9953 - val_loss: 0.0157 - val_acc: 0.9940

Epoch 00004: val_loss did not improve
Epoch 5/20
20000/20000 [==============================] - 1s 42us/step - loss

In [14]:
# 模型预测
model.load_weights('merge_fine_weight.h5')
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

12500/12500 [==============================] - 1s 42us/step


In [15]:
# 测试集预测
import os
import pandas as pd
from keras.preprocessing.image import *

# 获得当前目录
current_dir = os.getcwd()
df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(current_dir + "/data/test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/') + 1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('merged_pred.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


0.03843